In [104]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE, SelectKBest, f_regression

In [105]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [112]:
data = pd.read_excel('data/data_ford_price.xlsx')
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [107]:
# Выделим числовые признаки
data = data.select_dtypes(include='number')
# Удалим пропуски
data.dropna(inplace = True)
# Выделим целевой признак
y = data['price']
X = data.drop(columns='price')
# Разделим выборку на тренировочную  и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [108]:
# Обучим модель на всех признаках и посчитаем среднюю абсолютную ошибку
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_rfe = model.predict(X_test)
print('MAE для всех числовых признаков: {}'.format(round(mean_absolute_error(y_test, y_pred_rfe))))

MAE для всех числовых признаков: 4704


In [109]:
# Определим 3 наиболее значимых признака для линейной регрессии методом рекурсивного исключения признаков 
estimator = LinearRegression()
selector_1 = RFE(estimator, n_features_to_select=3, step=1)
selector_1 = selector_1.fit(X_train, y_train)
# Создадим список столбцов для обучения 
col_list_rfe = list(selector_1.get_feature_names_out())
col_list_rfe

['year', 'condition', 'cylinders']

In [110]:
# Определим 3 наиболее значимых признака на основе фильтров
selector_2 = SelectKBest(k=3)
selector_2.fit(X_train, y_train)
# Создадим список столбцов для обучения 
col_list_skb = list(selector_2.get_feature_names_out())
col_list_skb

['year', 'condition', 'odometer']

In [111]:
# Обучим линейную регрессию на 3 признаках, полученных методом рекурсивного исключения признаков и посчитаем среднюю абсолютную ошибку
model.fit(X_train[col_list_rfe], y_train)
y_pred_rfe = model.predict(X_test[col_list_rfe])
print('MAE для 3-х признаков RFE: {}'.format(round(mean_absolute_error(y_test, y_pred_rfe))))
# Обучим линейную регрессию на 3 признаках на основе фильтра

model.fit(X_train[col_list_skb], y_train)
y_pred_skb = model.predict(X_test[col_list_skb])
print('MAE для 3-х признаков SelectKBest: {}'.format(round(mean_absolute_error(y_test, y_pred_skb))))

MAE для 3-х признаков RFE: 5077
MAE для 3-х признаков SelectKBest: 4793


Абсолютная ошибка во втором случае меньше, чем в первом. Метод SelectKBest подобрал более важные признаки, чем метод рекурсивного исключения признаков.